# Geometric view of data

As seen earlier, every input is just a collection of values pertaining to every feature. Thus, we can view each feature as a seperate dimension. Now we can start performing geometric operations on these inputs like Eucledian distance.
Viewing input as vectors inherently assumes that features are not correlated to each other.

## Converting features into vectors

- Trivial in case of `real valued features`
- (0/1) in case of `binary features`
- v-many binary indicator features for `categorical features`

Explanation of last point: Suppose you want to convert a feature which conveys the color of a particular thing namely {RED, GREEN, BLUE, YELLOW}. If you map these values to {0,1,2,3} respectively, it conveys that RED is more similar to GREEN thann YELLOW which is not we wanted to say. All colors are equally dissimilar to each other. To convey this information, we need to convert a single valued feature into many-valued binary indicators namely {IsThisRed, IsThisGreen, IsThisBlue, IsThisYellow}.


# K-Nearest Neighbors

__Basic Idea__: Similar points have same labels.

Common measure of similarity is Euclidean distance
> $ \large{d(a,b) = [\sum^{D}_{d=1}(a_d - b_d)^2]^{1/2}}$

There is no training in this algorithm. We just save all the examples given to us and when we are given the task of prediction, we let its k-nearest neighbours vote for the label it should have.

__Inductive Bias__: Points with same label are always close to each other in term of euclidean distance.

Despite its simplicity, this algorithm is _frustatingly effective_.

## What value should K take?

K is what you call a __Hyperparameter__. This is a parameter which we have to decide apriori. This parameter cannot be learned through training on examples.

Depth of a decision tree is a hyperparameter. You have to decide on it before beginning to train because algorithm will always favour an overfitting model(i.e. a tree which will have all of its input data points on leaves) because it reduces the training error.

Here K is a hyperparameter. K denotes the number of nearest neighbors algorithm should look at to decide the output. This can only be figured out after training several times with different values of K and picking the model with minimum error on validation data.

Small value of K leads to overfitting and large value of K leads to underfitting. Why?

### Question: How does this algorithm treats features differently as compared to decision tree?

When you find distance between two points in space, you treat all dimensions equally. All dimensions have equal say in the distance. This implies that in KNN all features have equal importance. 

On the other hand, main purpose of DT algorithm is to find most valuable features and let them decide the output.

Based on above argument, if a dataset has only few relevant features and lots of irrelevant features, KNN will perform poorly.

## Decision Boundaries

A boundary between data points, derived from the trained model, which seperates points of different class. For eg. Suppose we have a line on a 2D plane above which all points have positive label and point with negative label below it. This line is the decision boundary derived from a simple linear model.

<img src="assets/Various decision boundaries - KDNuggets.png">

### KNN

<img src="assets/KNN decision boundary - stack exchange.png">

## Interactive Demos

- [KNN](https://lettier.com/projects/knearestneighbors/)
- [Decision Tree](http://www.r2d3.us/visual-intro-to-machine-learning-part-1/)

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import operator,random,math
from sklearn.neighbors import KNeighborsClassifier as KNC

%matplotlib inline

data src - https://www.kaggle.com/uciml/pima-indians-diabetes-database

In [2]:
diabetes_file_path = 'assets/diabetes.csv'
diabetes_data = pd.read_csv(diabetes_file_path)
del diabetes_data['SkinThickness']

In [3]:
diabetes_data.head()

,Pregnancies,Glucose,BloodPressure,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,0,33.6,0.627,50,1
1,1,85,66,0,26.6,0.351,31,0
2,8,183,64,0,23.3,0.672,32,1
3,1,89,66,94,28.1,0.167,21,0
4,0,137,40,168,43.1,2.288,33,1


In [4]:
diabetes_data.shape

(768, 8)

In [5]:
def split_train_test(x,y,ratio=0.8):
    count_train = int(x.shape[0]*ratio)
    train_x = np.array(x[:count_train])
    test_x = np.array(x[count_train:])
    train_y = np.array(y[:count_train])
    test_y = np.array(y[count_train:])
    return (train_x,test_x,train_y,test_y)

In [6]:
diabetes_data.sample(frac=1).reset_index(drop=True)
labels = diabetes_data['Outcome']
features = diabetes_data.drop(labels='Outcome',axis=1)
train_x,test_x,train_y,test_y = split_train_test(features,labels,0.85)
k = 5

In [7]:
def predict(x):
    labels = []
    for inp in x:
        # find distance of x from all points in training data
        distances = [(math.sqrt(np.square(tx-inp).sum()),ty) for tx,ty in zip(train_x,train_y)]
#         sort these distances
        distances.sort(key=operator.itemgetter(0))
        label = np.array([tup[1] for tup in distances[:k]]).sum()
        if label*2 >= k:
            labels.append(1)
        else:
            labels.append(0)
    return labels

In [8]:
local = predict(test_x)
accurate = 0
for i in range(test_x.shape[0]):
    if local[i] == test_y[i]:
        accurate+=1
print(accurate/test_x.shape[0])

0.75


In [9]:
model = KNC(n_neighbors=5,p=2,algorithm='brute')
model.fit(train_x,train_y)
lib = model.predict(test_x)
accurate = 0
for i in range(test_x.shape[0]):
    if lib[i] == test_y[i]:
        accurate+=1
print(accurate/test_x.shape[0])

0.75


In [10]:
%time predict(test_x)

CPU times: user 1.26 s, sys: 24.4 ms, total: 1.28 s
Wall time: 336 ms


[0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0]

Checking if the implementation has predicted any label which does not match with library function

In [11]:
any(x>0 for x in abs(local-lib))

False